In [2]:
colab = False

if colab == True:
    !pip install nnunetv2
    !pip install torchio
    

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [3]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
import shutil
from collections import OrderedDict
import json
import numpy as np
import nnunetv2

if colab:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)
    base_dir = '/content/drive/My Drive/Colab Notebooks/tumor-segmentation'
    os.chdir(base_dir)
    torch.cuda.empty_cache()

    !git clone https://github.com/MIC-DKFZ/nnUNet.git
    !git clone https://github.com/NVIDIA/apex

    training_size = 400


else:
    base_dir = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'
    training_size = 200

from utils import validate_segmentation, plot_prediction, dice_score
import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example

path_to_mask = 'data/patients/labels/'
path_to_imgs = 'data/patients/imgs/'
path_to_controls = 'data/controls/imgs/' #healthy individuals (no tumors)
dataset_path = 'data.zip'
dataset_dir_name = 'data'
dataset_dir = Path(dataset_dir_name)

if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url}
    !unzip -qq {dataset_path}

ModuleNotFoundError: No module named 'google'

In [3]:
images_dir = dataset_dir / 'patients/imgs'
labels_dir = dataset_dir / 'patients/labels'
controls_dir = dataset_dir / 'controls/imgs'
image_paths = sorted(images_dir.glob('*.png'))
label_paths = sorted(labels_dir.glob('*.png'))
control_paths = sorted(controls_dir.glob('*.png'))

dataset_ID = 1

dataset = torchio_compose_train(image_paths, label_paths, control_paths, 
                                cropsize=(400,991), train_size = training_size,
                                dataset_ID=dataset_ID)

Found 182 subjects


  0%|          | 0/181 [00:00<?, ?it/s]

Landmarks: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.33322500e-02 6.22397969e-01 9.23236779e+00
 1.72545437e+01 2.07381415e+01 2.53002322e+01 4.03121636e+01
 1.00000000e+02]
Augmenting 18 subjects


 17%|█▋        | 3/18 [00:01<00:08,  1.78it/s]/home/asp/anaconda3/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Input image is 2D, but "2" is in axes: (0, 1, 2)
  transformed = self.apply_transform(subject)
100%|██████████| 18/18 [00:09<00:00,  1.81it/s]


Saving 200 subjects to nnUNet_raw/Dataset001...


100%|██████████| 200/200 [00:02<00:00, 85.11it/s]


In [4]:
# Define the base path for your nnUNet directories
base_path = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'

# Set the environment variables
os.environ['nnUNet_raw'] = os.path.join(base_path, 'nnUNet_raw')
os.environ['nnUNet_preprocessed'] = os.path.join(base_path, 'nnUNet_preprocessed')
os.environ['nnUNet_results'] = os.path.join(base_path, 'nnUNet_results')

In [5]:
!nnUNetv2_plan_and_preprocess -h

usage: nnUNetv2_plan_and_preprocess [-h] [-d D [D ...]] [-fpe FPE]
                                    [-npfp NPFP] [--verify_dataset_integrity]
                                    [--no_pp] [--clean] [-pl PL]
                                    [-gpu_memory_target GPU_MEMORY_TARGET]
                                    [-preprocessor_name PREPROCESSOR_NAME]
                                    [-overwrite_target_spacing OVERWRITE_TARGET_SPACING [OVERWRITE_TARGET_SPACING ...]]
                                    [-overwrite_plans_name OVERWRITE_PLANS_NAME]
                                    [-c C [C ...]] [-np NP [NP ...]]
                                    [--verbose]

options:
  -h, --help            show this help message and exit
  -d D [D ...]          [REQUIRED] List of dataset IDs. Example: 2 4 5. This
                        will run fingerprint extraction, experiment planning
                        and preprocessing for these datasets. Can of course
                        al

In [6]:
!nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity

Fingerprint extraction...
Dataset001
Using <class 'nnunetv2.imageio.natural_image_reager_writer.NaturalImage2DIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.natural_image_reager_writer.NaturalImage2DIO'> as reader/writer
100%|█████████████████████████████████████████| 200/200 [00:05<00:00, 34.90it/s]
Experiment planning...
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 8, 'patch_size': array([384, 512]), 'median_image_size_in_voxels': array([333., 488.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': (2, 2, 2, 2, 2, 2, 2), 'n_conv_per_stage_decoder': (2, 2, 2, 2, 2, 2), 'num_pool_per_axis': [6, 6], 'pool_op_k

In [7]:
!nnUNetv2_train -h

usage: nnUNetv2_train [-h] [-tr TR] [-p P]
                      [-pretrained_weights PRETRAINED_WEIGHTS]
                      [-num_gpus NUM_GPUS] [--use_compressed] [--npz] [--c]
                      [--val] [--val_best] [--disable_checkpointing]
                      [-device DEVICE]
                      dataset_name_or_id configuration fold

positional arguments:
  dataset_name_or_id    Dataset name or ID to train with
  configuration         Configuration that should be trained
  fold                  Fold of the 5-fold cross-validation. Should be an int
                        between 0 and 4.

options:
  -h, --help            show this help message and exit
  -tr TR                [OPTIONAL] Use this flag to specify a custom trainer.
                        Default: nnUNetTrainer
  -p P                  [OPTIONAL] Use this flag to specify a custom plans
                        identifier. Default: nnUNetPlans
  -pretrained_weights PRETRAINED_WEIGHTS
                        [O

## The following only works with CUDA

In [8]:
!nnUNetv2_train Dataset001 2d 0

Using device: cuda:0
/home/asp/anaconda3/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Traceback (most recent call last):
  File "/home/asp/anaconda3/bin/nnUNetv2_train", line 8, in <module>
    sys.exit(run_training_entry())
             ^^^^^^^^^^^^^^^^^^^^
  File "/home/asp/anaconda3/lib/python3.11/site-packages/nnunetv2/run/run_training.py", line 268, in run_training_entry
    run_train